# 3. 신경망
* * *

+ 퍼셉트론으로 복잡한 함수도 표현할 수 있음
+ 그러나, 가중치를 설정하는 작업 ( 원하는 결과를 출력하도록 가중치 값을 적절히 정하는 작업 ) 은 여전히 사람이 수동으로 해야함
</br></br>
+ 신경망의 중요한 성질 == 가중치 매개변수의 적절한 값을 데이터로부터 잗ㅇ으로 학습하는 능력

## 3.5 출력층 설계하기
+ 신경망은 분류와 회귀 모두에 이용할 수 있음
    + 출력층에서 사용하는 활성화 함수가 다름
    + <spen style="color: gold;">**분류 ( classification )**</spen> == 데이터가 어느 클래스 ( class ) 에 속하느냐는 문제
    + <spen style="color: gold;">**회귀 ( regression )**</spen> == 입력 데이터에서 ( 연속적인 ) 수치를 예측하는 문제

##### <br><br><span style="color: mediumaquamarine;">**항등 함수와 소프트 맥스 함수 구현하기**</spen>

<spen style="color: gold;">**항등 함수**</spen>
+ 항등 함수 ( identity function ) == 입력을 그대로 출력
    + 입력 == 출력
+ 항등 함수에 의한 변환은 은닉층에서의 활성화 함수와 마찬가지로 화살표로 그림

<img src="../img/identity_function.png" width='300'>
<br>&nbsp;<br>
<br>&nbsp;<br>

<spen style="color: gold;">**소프트맥스  함수**</spen>  

<img src="../img/softmax_function2.png" width='300'>

+ $exp(x)$ == $e^x$를 뜻하는 지수함수 ( exponential function ) ( $e$ == 자연상수 )
+ $n$ == 출력층의 뉴런 수
+ $y_k$ == 출력층의 $k$번째 출력
+ 소프트맥스 함수 ( softmax function )
    + 분자 == 입력 신호 $a_k$의 지수 함수
    + 분모 == 모든 입력 신호의 지수 함수의 합

<br>&nbsp;<br>
<img src="../img/softmax_function1.png" width='300'>

+ 소프트맥스틔 출력은 모든 입력 신호로부터 화살표를 받음
    + <spen style="color: palevioletred;">**출력층의 각 뉴런이 모든 입력 신호에서 영향을 받기 때문**</spen>

In [1]:
import numpy as np
a = np.array([0.3, 2.9, 4.0])
exp_a = np.exp(a) # 지수 함수
print(exp_a)

sum_exp_a = np.sum(exp_a) # 지수 함수의 합
print(sum_exp_a)

y = exp_a / sum_exp_a
print(y)

[ 1.34985881 18.17414537 54.59815003]
74.1221542101633
[0.01821127 0.24519181 0.73659691]


In [2]:
def softmax(a):
    exp_a = np.exp(a)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a

    return y

##### <br><br><span style="color: mediumaquamarine;">**소프트맥스 함수 구현 시 주의점**</spen>
+ 앞의 식에서 <spen style="color: gold;">**오버플로 ( overflow )**</spen> 문제 발생
    + 소프트맥스 함수는 지수 함수를 사용하는데, 지수 함수는 쉽게 아주 큰 값을 내뱉음
    + 큰 값끼리 나눗셈을 하면 결과 수치가 '불안정'해짐

소프트맥스 함수 개선한 수식  
<img src="../img/softmax_function3.png" width='500'>

+ <spen style="color: palevioletred;">**소프트맥스의 지수 함수를 계산할 때 어떤 정수를 더해도 ( 혹은 빼도 ) 결과는 바뀌지 않는다.**</spen>
+ $C'$ == 입력 신호 중 최댓값을 이용하는 것이 일반적임

In [3]:
a = np.array([1010, 1000, 990])
print(np.exp(a) / np.sum(np.exp(a))) # 소프트맥스 함수의 계산 <- 제대로 계산되지 않음
c = np.max(a)   # c = 1010 ( 최댓값 )
print(a - c)
print(np.exp(a - c) / np.sum(np.exp(a - c)))

[nan nan nan]
[  0 -10 -20]
[9.99954600e-01 4.53978686e-05 2.06106005e-09]


C:\Users\gahyu\AppData\Local\Temp\ipykernel_29276\1162427212.py:2: RuntimeWarning: overflow encountered in exp
  print(np.exp(a) / np.sum(np.exp(a))) # 소프트맥스 함수의 계산 <- 제대로 계산되지 않음
C:\Users\gahyu\AppData\Local\Temp\ipykernel_29276\1162427212.py:2: RuntimeWarning: invalid value encountered in divide
  print(np.exp(a) / np.sum(np.exp(a))) # 소프트맥스 함수의 계산 <- 제대로 계산되지 않음


+ 아무런 조치 없이 그냥 계산하면 ```nan``` 출력
+ 입력 신호 중 <spen style="color: palevioletred;">**최댓값을 빼주면 올바르게 계산**</spen>

In [4]:
def softmax(a):
    c = np.max(a)
    exp_a = np.exp(a - c) # 오버플로 대책
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a

    return y

##### <br><br><span style="color: mediumaquamarine;">**소프트맥스 함수의 특징**</spen>

+ 소프트맥스 함수의 출력 == 0에서 1.0 사이의 실수
+ 소프트맥스 함수 출력의 총합 == 1
    + 이 성질을 통해 소프트맥스 함수의 출력을 <spen style="color: rosybrown;">**확률**</spen>로 해석할 수 있다.

In [5]:
a = np.array([0.3, 2.9, 4.0])
y = softmax(a)
print("y = ", y)
print(np.sum(y))

y =  [0.01821127 0.24519181 0.73659691]
1.0


+ y[0] == 0.018 ( 18% ), y[1] == 0.245 ( 24.5% ), y[2] = 0.737 ( 73.7% )  
    <spen style="color: gold;">**결과**</spen>
    + 2번째 원소의 확률이 가장 높으니 답은 2번째 클래스다.
    + 74% 확률로 2번째 클래스, 25%의 확률로 1번째 클래스, 1%의 확률로 0번째 클래스다.
+ <spen style="color: palevioletred;">**소프트맥스 함수를 이용함으로써 문제를 확률적 ( 통계적 ) 으로 대응할 수 있게 되는 것이다.**</spen>

+ 소프트맥스 함수를 적용해도 각 원소의 대소 관계가 변하지 않음
    + 지수 함수 $y = exp(x)$가 단조 증가 함수이기 때문
        + 단조 증가 함수 == 정의역 원소 $a, b$가 $a \leq b$일 때, $f(a) \leq f(b)$가 성립하는 함수
+ <spen style="color: palevioletred;">**신경망으로 분류할 때는 출력층의 소프트맥스 함수를 생략해도 된다.**</spen>
    + 현업에서도 지수 함수 계산에 드는 자원 낭비를 줄이고자 출력층의 소프트맥스 함수를 생략해도 됨  
    <spen style="color: gold;">**근거**</spen>
        + 신경망을 이용한 분류에서는 일반적으로 가장 큰 출력을 내는 뉴런에 해당하는 클래스만 인식함
        + 소프트맥스 함수를 적용해도 출력이 가장 큰 뉴런의 위치는 달라지지 않음

기계학습의 문제 풀이는 학습과 추론(inference)의 두 단계를 거쳐 이뤄진다.  
학습 단계에서 모델을 학습하고, 추론 단계에서 앞서 학습한 모델로 미지의 데이터에 대해서 추론(분류)을 수행한다.  
위에서 설명한 대로, 추론 단계에서는 출력층의 소프트맥스 함수를 생략하는 것이 일반적이지만, 신경망을 학습시킬 때는 출력층에서 소프트맥스 함수를 사용한다.

##### <br><br><span style="color: mediumaquamarine;">**출력층의 뉴런 수 정하기**</spen>
+ 출력층의 뉴런 수는 풀려는 문제에 맞게 적절히 정해야함
+ 분류에서는 분류하고 싶은 클래스 수로 설정하는 것이 일반적

출력층의 뉴런은 각 숫자에 대응한다.  
<img src="../img/neuron_number.png" width='600'>

+ 뉴런의 회색 농도가 해당 뉴런의 출력 값의 크기를 의미함
+ 색이 가장 짙은 $y_2$ 뉴런이 가장 큰 값을 출력
+ 이 신경망이 선택한 클래스는 $y_2$, 즉 입력 이미지를 숫자 '2'로 판단했음을 의미함